In [1]:
import aqt

In [2]:
browser = aqt.dialogs._dialogs['Browser'][1]

In [3]:
table = browser.table

In [4]:
table.mode()

'R'

In [5]:
model = table._model

In [6]:
model._items[:10]

[1659295892837,
 1659295895775,
 1659295898290,
 1659295928602,
 1659295943381,
 1659295945634,
 1659295947053,
 1659295949476,
 1659295950522,
 1659295951504]

In [7]:
model._items

[1659295892837, 1659295895775, 1659295898290, 1659295928602, 1659295943381, 1659295945634, 1659295947053, 1659295949476, 1659295950522, 1659295951504, 1659295952456, 1659295955709, 1659295959884, 1659295961823, 1659295964024, 1659295966193, 1659295967950, 1659295969445, 1659295976845, 1659295979256, 1659295981353, 1659295983224, 1659295988649, 1659296313152, 1659296903686, 1659297358706, 1659298321844, 1659300480214, 1659326474000, 1659408173000, 1659408185000, 1659408195000, 1659408203000, 1659408208000, 1659408215000, 1659408217000, 1669072141900, 1669072186506, 1669073314428, 1669075184776, 1669076251929, 1669076589200, 1669078751019, 1672427326854]

In [8]:
model._state.get_review(model._items[9]).tags

[]

In [9]:
table.get_current_review()

id: 1659295892837
cid: 1659061121013
usn: 1741
button_chosen: 4
interval: 3
last_interval: -60
ease_factor: 2500
taken_millis: 17943
mtime_secs: 1659295892

In [10]:
import pandas as pd

On the left axis I want note tags beneath a hierarchy, and on the top axis review tags beneath a hierarchy.

The vector for a particular note tag should coalesce: For all notes within the tag, for all cards within the note, for all reviews within the card, for all review tags within the review.

Needs:

- A single number for each review tag.
    - Let's start with: for each review tag, the proportion of reviews with that tag.
- Comparabillity. This means accounting for variations in the number of notes, cards, reviews. This suggests averaging.
- Decaying average. More recent reviews should have higher weight.

All notes, cards, and reviews in the browser:

In [11]:
reviews = [model._state.get_review(item) for item in model._items]
cards = [model._state.get_card(item) for item in model._items]
notes = [model._state.get_note(item) for item in model._items]

In [12]:
notes

In [14]:
reviews

[id: 1659295892837
 cid: 1659061121013
 usn: 1741
 button_chosen: 4
 interval: 3
 last_interval: -60
 ease_factor: 2500
 taken_millis: 17943
 mtime_secs: 1659295892,
 id: 1659295895775
 cid: 1659061207486
 usn: 1741
 button_chosen: 4
 interval: 3
 last_interval: -60
 ease_factor: 2500
 taken_millis: 2920
 mtime_secs: 1659295895,
 id: 1659295898290
 cid: 1659061306420
 usn: 1741
 button_chosen: 4
 interval: 2
 last_interval: -60
 ease_factor: 2500
 taken_millis: 2498
 mtime_secs: 1659295898,
 id: 1659295928602
 cid: 1659061591875
 usn: 1963
 button_chosen: 1
 interval: -60
 last_interval: -60
 taken_millis: 11454
 mtime_secs: 1672949983,
 id: 1659295943381
 cid: 1659111295226
 usn: 1741
 button_chosen: 4
 interval: 2
 last_interval: -60
 ease_factor: 2500
 taken_millis: 14744
 mtime_secs: 1659295943,
 id: 1659295945634
 cid: 1659111609526
 usn: 1741
 button_chosen: 4
 interval: 3
 last_interval: -60
 ease_factor: 2500
 taken_millis: 2237
 mtime_secs: 1659295945,
 id: 1659295947053
 cid:

In [15]:
import itertools

Note tags represented in the browser.

In [68]:
note_tags = list(filter(lambda tag: tag.startswith('ECET-') or tag.startswith('CET-'), set(itertools.chain(*(note.tags for note in notes)))))
note_tags

['CET-05.05-TheSubstitutionRule',
 'ECET-06.01-IntegrationByParts',
 'ECET-06.02-TrigonometricIntegralsAndSubstitutions',
 'ECET-06.03-PartialFractions2',
 'CET-07.04-IntegrationOfRationalFunctionsByPartialFractions']

Review tags represented in the browser.

In [43]:
review_tags = list(filter(lambda tag: tag.startswith('.feedback::trouble::'), set(itertools.chain(*(review.tags for review in reviews)))))
review_tags

['.feedback::trouble::procedural::parts::skipped-chain-rule',
 '.feedback::trouble::procedural::substitution::skipped-chain-rule',
 '.feedback::trouble::procedural::partial-fractions::conversion-to-common-denominator',
 '.feedback::trouble::algebra::wrong-polynomial-power',
 '.feedback::trouble::algebra::omission::dropped-sign',
 '.feedback::trouble::transcription::sign-flip',
 '.feedback::trouble::algebra::conflation::symbol-swap',
 '.feedback::trouble::algebra::conflation::addition-and-subtraction',
 '.feedback::trouble::algebra::polynomial-expansion',
 '.feedback::trouble::algebra::conflation::expressions',
 '.feedback::trouble::procedural::integration::wrong-variable',
 '.feedback::trouble::algebra::omission::dropped-factor',
 '.feedback::trouble::algebra::omission::dropped-power',
 '.feedback::trouble::transcription::miscopied',
 '.feedback::trouble::algebra::conflation::fraction-inversion',
 '.feedback::trouble::algebra::conflation::multiplication-and-division',
 '.feedback::trou

In [18]:
review_ids = [review.id for review in reviews]
review_ids

[1659295892837,
 1659295895775,
 1659295898290,
 1659295928602,
 1659295943381,
 1659295945634,
 1659295947053,
 1659295949476,
 1659295950522,
 1659295951504,
 1659295952456,
 1659295955709,
 1659295959884,
 1659295961823,
 1659295964024,
 1659295966193,
 1659295967950,
 1659295969445,
 1659295976845,
 1659295979256,
 1659295981353,
 1659295983224,
 1659295988649,
 1659296313152,
 1659296903686,
 1659297358706,
 1659298321844,
 1659300480214,
 1659326474000,
 1659408173000,
 1659408185000,
 1659408195000,
 1659408203000,
 1659408208000,
 1659408215000,
 1659408217000,
 1669072141900,
 1669072186506,
 1669073314428,
 1669075184776,
 1669076251929,
 1669076589200,
 1669078751019,
 1672427326854]

In [19]:
rt0 = review_tags[0]
rt0

'.feedback::trouble::procedural::parts::skipped-chain-rule'

In [20]:
pd.DataFrame({rt:[rt in review.tags for review in reviews] for rt in review_tags})

,.feedback::trouble::procedural::parts::skipped-chain-rule,.feedback::technique::factor-counting,.feedback::trouble::procedural::substitution::skipped-chain-rule,.feedback::technique::substitution,.feedback::trouble::procedural::partial-fractions::conversion-to-common-denominator,.feedback::trouble::algebra::wrong-polynomial-power,.feedback::technique::intermediate-substitution,.feedback::trouble::algebra::omission::dropped-sign,ann:tr,.feedback::trouble::transcription::sign-flip,...,.feedback::trouble::algebra::conflation::expressions,.feedback::trouble::procedural::integration::wrong-variable,.feedback::trouble::algebra::omission::dropped-factor,.feedback::trouble::algebra::omission::dropped-power,.feedback::technique::completingTheSquare,ann:verify,.feedback::trouble::transcription::miscopied,.feedback::trouble::algebra::conflation::fraction-inversion,.feedback::trouble::algebra::conflation::multiplication-and-division,.feedback::trouble::algebra::conflation::sign-flip
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
note_tags

['CET-05.05-TheSubstitutionRule',
 'ECET-06.01-IntegrationByParts',
 'ann:hard',
 'ECET-06.02-TrigonometricIntegralsAndSubstitutions',
 'ECET-06.03-PartialFractions2',
 'ann:technique',
 'CET-07.04-IntegrationOfRationalFunctionsByPartialFractions']

This produces a list of trouble points for each note tag visible in the browser. Very simple: it lists all trouble tags for any reviews corresponding to the note tag.

In [22]:
{note_tag:list(
        set(
            itertools.chain(
                *(
                    filter(
                        lambda review_tag: review_tag.startswith('.feedback::trouble::'),
                        model._state.get_review(item).tags
                    )
                    for item in model._items
                    if note_tag in model._state.get_note(item).tags
                )
            )
        )
    )
    for note_tag in filter(
        lambda note_tag: note_tag.startswith('ECET-') or note_tag.startswith('CET-'),
        note_tags
    )
}

{'CET-05.05-TheSubstitutionRule': ['.feedback::trouble::algebra::omission::dropped-factor'],
 'ECET-06.01-IntegrationByParts': [],
 'ECET-06.02-TrigonometricIntegralsAndSubstitutions': ['.feedback::trouble::algebra::conflation::addition-and-subtraction',
  '.feedback::trouble::procedural::parts::skipped-chain-rule',
  '.feedback::trouble::procedural::substitution::skipped-chain-rule',
  '.feedback::trouble::transcription::sign-flip',
  '.feedback::trouble::algebra::omission::dropped-power',
  '.feedback::trouble::transcription::miscopied',
  '.feedback::trouble::algebra::conflation::multiplication-and-division',
  '.feedback::trouble::algebra::conflation::sign-flip'],
 'ECET-06.03-PartialFractions2': ['.feedback::trouble::algebra::conflation::expressions',
  '.feedback::trouble::procedural::partial-fractions::conversion-to-common-denominator',
  '.feedback::trouble::transcription::miscopied'],
 'CET-07.04-IntegrationOfRationalFunctionsByPartialFractions': ['.feedback::trouble::algebra:

In [23]:
notes[0]

In [28]:
model.col.get_revlog_entry

In [35]:
note.fields

['CET-07.04-Xm02', '', '220801-2025: trouble: arithmetic:']

In [37]:
import datetime

In [42]:
datetime.datetime.fromtimestamp(review.id/1000)

datetime.datetime(2022, 12, 30, 12, 8, 46, 854000)

In [44]:
review_tags = list(filter(lambda tag: tag.startswith('.feedback::trouble::'), set(itertools.chain(*(review.tags for review in reviews)))))
review_tags

['.feedback::trouble::procedural::parts::skipped-chain-rule',
 '.feedback::trouble::procedural::substitution::skipped-chain-rule',
 '.feedback::trouble::procedural::partial-fractions::conversion-to-common-denominator',
 '.feedback::trouble::algebra::wrong-polynomial-power',
 '.feedback::trouble::algebra::omission::dropped-sign',
 '.feedback::trouble::transcription::sign-flip',
 '.feedback::trouble::algebra::conflation::symbol-swap',
 '.feedback::trouble::algebra::conflation::addition-and-subtraction',
 '.feedback::trouble::algebra::polynomial-expansion',
 '.feedback::trouble::algebra::conflation::expressions',
 '.feedback::trouble::procedural::integration::wrong-variable',
 '.feedback::trouble::algebra::omission::dropped-factor',
 '.feedback::trouble::algebra::omission::dropped-power',
 '.feedback::trouble::transcription::miscopied',
 '.feedback::trouble::algebra::conflation::fraction-inversion',
 '.feedback::trouble::algebra::conflation::multiplication-and-division',
 '.feedback::trou

In [45]:
rtags = {}

In [48]:
rtags.setdefault('fubar', 0)

0

In [49]:
rtags.setdefault('fubar', 1)

0

This is giving for each note a decaying average of review-tag scores.

In [64]:
empty_rtag_scores = {rtag:0 for rtag in review_tags}
empty_rtag_scores

{'.feedback::trouble::procedural::parts::skipped-chain-rule': 0,
 '.feedback::trouble::procedural::substitution::skipped-chain-rule': 0,
 '.feedback::trouble::procedural::partial-fractions::conversion-to-common-denominator': 0,
 '.feedback::trouble::algebra::wrong-polynomial-power': 0,
 '.feedback::trouble::algebra::omission::dropped-sign': 0,
 '.feedback::trouble::transcription::sign-flip': 0,
 '.feedback::trouble::algebra::conflation::symbol-swap': 0,
 '.feedback::trouble::algebra::conflation::addition-and-subtraction': 0,
 '.feedback::trouble::algebra::polynomial-expansion': 0,
 '.feedback::trouble::algebra::conflation::expressions': 0,
 '.feedback::trouble::procedural::integration::wrong-variable': 0,
 '.feedback::trouble::algebra::omission::dropped-factor': 0,
 '.feedback::trouble::algebra::omission::dropped-power': 0,
 '.feedback::trouble::transcription::miscopied': 0,
 '.feedback::trouble::algebra::conflation::fraction-inversion': 0,
 '.feedback::trouble::algebra::conflation::mu

In [55]:
weight = 0.65
decay = 1 - weight

In [72]:
empty_rtag_score_averaging = {rtag:[] for rtag in review_tags}

In [74]:
ntag_rtag_score_averaging = {ntag:empty_rtag_score_averaging.copy() for ntag in note_tags}

In [ ]:
ntag_rtag_score_averaging

{'CET-05.05-TheSubstitutionRule': {'.feedback::trouble::procedural::parts::skipped-chain-rule': [],
  '.feedback::trouble::procedural::substitution::skipped-chain-rule': [],
  '.feedback::trouble::procedural::partial-fractions::conversion-to-common-denominator': [],
  '.feedback::trouble::algebra::wrong-polynomial-power': [],
  '.feedback::trouble::algebra::omission::dropped-sign': [],
  '.feedback::trouble::transcription::sign-flip': [],
  '.feedback::trouble::algebra::conflation::symbol-swap': [],
  '.feedback::trouble::algebra::conflation::addition-and-subtraction': [],
  '.feedback::trouble::algebra::polynomial-expansion': [],
  '.feedback::trouble::algebra::conflation::expressions': [],
  '.feedback::trouble::procedural::integration::wrong-variable': [],
  '.feedback::trouble::algebra::omission::dropped-factor': [],
  '.feedback::trouble::algebra::omission::dropped-power': [],
  '.feedback::trouble::transcription::miscopied': [],
  '.feedback::trouble::algebra::conflation::fractio

In [77]:
for note in notes:
    print(f"note ID: {note.id}, field 0: {note.fields[0]}")
    rids = model._state.review_ids_from_note_ids([note.id,])
    #print(rids)
    rids.sort()
    rtag_scores = empty_rtag_scores.copy()
    for rid in rids:
        review = model.col.get_revlog_entry(rid)
        for tag in filter(lambda tag: tag.startswith('.feedback::trouble::'), review.tags):
            rtag_score = rtag_scores[tag]
            rtag_score = (decay*rtag_score) + (weight)
            rtag_scores[tag] = rtag_score
        print(f"review ID: {review.id}, tags: {review.tags}")
    all_rtag_scores[note.id] = rtag_scores
    print(f"rtag scores: {all_rtag_scores[note.id]}")
#    for ntag in note.tags:
#        for rtag, score in 

note ID: 1659061121012, field 0: CET-05.05-Xm01
review ID: 1659295892837, tags: []
rtag scores: {'.feedback::trouble::procedural::parts::skipped-chain-rule': 0, '.feedback::trouble::procedural::substitution::skipped-chain-rule': 0, '.feedback::trouble::procedural::partial-fractions::conversion-to-common-denominator': 0, '.feedback::trouble::algebra::wrong-polynomial-power': 0, '.feedback::trouble::algebra::omission::dropped-sign': 0, '.feedback::trouble::transcription::sign-flip': 0, '.feedback::trouble::algebra::conflation::symbol-swap': 0, '.feedback::trouble::algebra::conflation::addition-and-subtraction': 0, '.feedback::trouble::algebra::polynomial-expansion': 0, '.feedback::trouble::algebra::conflation::expressions': 0, '.feedback::trouble::procedural::integration::wrong-variable': 0, '.feedback::trouble::algebra::omission::dropped-factor': 0, '.feedback::trouble::algebra::omission::dropped-power': 0, '.feedback::trouble::transcription::miscopied': 0, '.feedback::trouble::algebra:

In [25]:
def trouble_by_note_tag():
    notes = [model._state.get_note(item) for item in model._items]
    note_tags = list(set(itertools.chain(*(note.tags for note in notes))))

In [61]:
filter?

Init signature: filter(self, /, *args, **kwargs)
Docstring:     
filter(function or None, iterable) --> filter object

Return an iterator yielding those items of iterable for which function(item)
is true. If function is None, return the items that are true.
Type:           type
Subclasses:     


In [26]:
{tag:[note for note in notes if tag in note.tags] for tag in note_tags}

{'CET-05.05-TheSubstitutionRule': [<anki.notes.Note at 0x7fe8b01d5820>,
 'ECET-06.01-IntegrationByParts': [<anki.notes.Note at 0x7fe8b01d58e0>,
 'ann:hard': [<anki.notes.Note at 0x7fe8b01d5ac0>],
 'ECET-06.02-TrigonometricIntegralsAndSubstitutions': [<anki.notes.Note at 0x7fe8b01d5940>,
 'ECET-06.03-PartialFractions2': [<anki.notes.Note at 0x7fe8b01d5250>,
 'ann:technique': [<anki.notes.Note at 0x7fe8b01d5820>,
 'CET-07.04-IntegrationOfRationalFunctionsByPartialFractions': [<anki.notes.Note at 0x7fe8b01d5e50>,
  <anki.notes.Note at 0x7fe752504130>]}